# Demo for MedAgent - First answer generation with naive RAG pipeline

This is the manual testing playground to test some basic workflows later properly implemented in the MedAgent repository.

This file is responsible for a first test of answer generation with naive retrieval (basically creating the second baseline for our system test). This means, for the question first the most similar chunks from the guidelines are retrieved, and then provided to a generator with the original question. For this setup, new feedback must be gathered and the results analyzed and visualized.

In [ ]:
# SETUP
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.local-env")
BACKEND_API_URL = "http://host.docker.internal:5000/api"